In [ ]:
%run "Подготовка данных.ipynb"

В ноутбуке "Подготовка данных" необходимо выбрать целевую переменную food_goods или nonfood_goods. После этого нужно сохранить ноутбук. В ходе этого ноутбука будет еще одна возможность выбрать переменную для рассмотрения. См. комментарии к коду.

In [ ]:
var_name

In [ ]:
def result(X, y):
    t = X.isnull().sum(axis = 0).max()
    y = y[t:]            
    X = X[t:] 
    X = X.reset_index(drop = True)
    y = y.reset_index(drop = True)
    X = sm.add_constant(X)
    n = round(len(X)*0.6)
    return X, y, n, t

In [ ]:
variables = [tab_2014['ipd'], tab_2014['ipd-1'], tab_2014['ipd-2'], tab_2014['ipd-3'],
             tab_2014['oil'], tab_2014['oil-1'], tab_2014['oil-2'],tab_2014['oil-3'],
             tab_2014['rvi'], tab_2014['rvi-1'], tab_2014['rvi-2'],tab_2014['rvi-3'],
             tab_2014['real_rate'], tab_2014['real_rate-1'],tab_2014['real_rate-2'],tab_2014['real_rate-3'],
             tab_2014[var_name+'-1'], tab_2014[var_name+'-2'], tab_2014[var_name+'-3']]

In [ ]:
combs = []
for i in range(1, 4):
    els = [list(x) for x in itertools.combinations(variables, i)]
    combs.extend(els)
len(combs)

In [ ]:
inf = []
for m in range(len(combs)):
    X = pd.DataFrame(combs[m]).T
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    
    start = results.fittedvalues.index[-1]
    predictions = []
    for i in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[i:i+1])
        predictions.append(a[i])
        model = sm.OLS(y[:i], X[:i])
        results = model.fit(cov_type='HC3')

    columns = X.columns
    r = results.rsquared_adj
    p = results.pvalues
    k=[]
    for i in range(0,len(p)):
        if p[i] < 0.05:
            k.append(p.index[i])
    MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)         
    inf.append([list(X.columns), k, r, MAE])
        
ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
p_values = []
regressors = []
for i in range(len(ols_all)):
    p_values.append(", ".join(ols_all['Значимые регрессоры'][i])) 
    regressors.append(", ".join(ols_all['Регрессоры'][i])) 
ols_all['Значимые регрессоры'] = p_values
ols_all['Регрессоры'] = regressors

df = pd.DataFrame()
for j in range(len(ols_all['Значимые регрессоры'])):
    if ', '.join(ols_all['Регрессоры'][j].split(', ')[1:]) in ols_all['Значимые регрессоры'][j]:
        df = df.append(ols_all[ols_all.index == j])
models = df.sort_values('Скорректированный R^2', ascending=False).reset_index(drop=True)[:10]

In [ ]:
if var_name == 'nonfood_goods':
    goods = neprod_detrend ## Выбираем тренды к целевой переменной
else: 
    goods = prod_detrend 

combs = []
for i in range(1, 3):
    els = [list(x) for x in itertools.combinations(goods, i)]
    combs.extend(els)

In [ ]:
s = pd.DataFrame()
for g in range(len(combs)):
    inf = []
    for m in range(len(models)):
        splited = models['Регрессоры'][m].split(', ')
        try:
            X1 = tab_2014.filter([splited[1], splited[2], splited[3]])
        except:
            try:
                X1 = tab_2014.filter([splited[1], splited[2]])
            except:
                X1 = tab_2014.filter([splited[1]])   
        X = X1.join(goods[combs[g]], how = 'left')
        y = tab_2014.filter([var_name])
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')
        start = results.fittedvalues.index[-1]
        predictions = []
        for i in range(start, y.index[-1]):
            ## добавляем прогнозное наблюдение
            a = results.predict(X[i:i+1])
            predictions.append(a[i])
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')
        columns = X.columns
        p = results.pvalues
        k=[]
        for i in range(0,len(p)):
            if p[i] < 0.05:
                k.append(p.index[i])
        r = results.rsquared_adj
        MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)
        inf.append([list(X.columns), k, r, MAE])    
    ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
    table = models.join(ols_all, how = 'left', rsuffix=' (тренды)')
    table['Разница MAE'] = table['MAE (тренды)'] - table['MAE']
    table['Разница R^2'] = table['Скорректированный R^2 (тренды)'] - table['Скорректированный R^2']
    table1 = table[(table['Разница MAE']<0) & (table['Разница R^2']>0)].reset_index(drop = True)
    for j in range(len(table1['Значимые регрессоры (тренды)'])):
        if combs[g][0] in table1['Значимые регрессоры (тренды)'][j]:
            s = s.append(table1[table1.index == j])

In [ ]:
len(s)

Здесь мы сортируем результаты по разнице R^2 и по разнице МАЕ. 

In [ ]:
s_r2_1 = s.sort_values(by='Разница R^2', ascending=False).reset_index(drop = True)[:10]
s_mae_1 = s.sort_values(by='Разница MAE', ascending=True).reset_index(drop = True)[:10]

В следующей ячейке необходимо выбрать таблицу, по которой будет происходить отрисовка графиков.

In [ ]:
variable = s_r2_1

In [ ]:
predictions_total_bench = []
predictions_total_gt = []

r_bench = []
r_gt = []

for i in range(len(variable)):
    X = tab_2014.filter(variable.iloc[:,0][i].split())
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    predictions = []
    r_sq = []
    start = results.fittedvalues.index[-1]
    for k in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[k:k+1])
        predictions.append(a[k])
        model = sm.OLS(y[:k], X[:k])
        results = model.fit(cov_type='HC3')
        r = results.rsquared_adj
        r_sq.append(r)
    
    predictions_total_bench.append(predictions)
    r_bench.append(r_sq)
    
    X = tab_2014.filter(variable.iloc[:,0][i].split()).join(goods[variable.iloc[:,4][i][len(variable.iloc[:,0][i].split()):]], how = 'left')
    y = tab_2014.filter([var_name])
    X, y, n, t = result(X, y)
    model = sm.OLS(y[:n], X[:n])
    results = model.fit(cov_type='HC3')
    start = results.fittedvalues.index[-1]
    predictions1 = []
    r_sq1 = []
    for k in range(start, y.index[-1]):
        ## добавляем прогнозное наблюдение
        a = results.predict(X[k:k+1])
        predictions1.append(a[k])
        model = sm.OLS(y[:k], X[:k])
        results = model.fit(cov_type='HC3')
        r = results.rsquared_adj
        r_sq1.append(r)
        
    predictions_total_gt.append(predictions1)
    r_gt.append(r_sq1)
    
mean_gt = pd.DataFrame(predictions_total_gt).mean(axis = 0)
mean_bch = pd.DataFrame(predictions_total_bench).mean(axis = 0)

mean_r_bench = pd.DataFrame(r_bench).mean(axis = 0)
mean_r_gt = pd.DataFrame(r_gt).mean(axis = 0)

plotly.offline.init_notebook_mode()

for i in range(len(variable.MAE)):

    if variable.MAE[i] == s_mae_1.MAE[i]:
        data = [plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=y.iloc[:,0][n:], mode='lines', name = 'Непродовольственный ретейл'),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=mean_bch, mode='lines', name = 'Усредненные прогнозы базовых моделей'),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                         y=mean_gt, mode='lines', name = 'Усредненные прогнозы расширенных моделей')]
    else:
        data = [plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                          y=mean_r_bench, mode='lines', name = 'Усредненный коэффициент детерминации базовых моделей', line=dict(color="red")),
               plotly.graph_objs.Scatter(x=prod[132+n+t:].index, 
                                         y=mean_r_gt, mode='lines', name = 'Усредненный коэффициент детерминации расширенных моделей', line=dict(color="#10CDAC"))]


layout = go.Layout(legend=dict(orientation="h"),font=dict(size=15,),autosize=False, width=1000,height=700,
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', xaxis = dict(showticklabels = True),
    yaxis = dict(showgrid = True, gridcolor = '#e3e3e3', gridwidth = 1, zerolinecolor = '#969696', zerolinewidth = 2,
    linecolor = '#636363', linewidth = 1, showticklabels = True))

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig)  

In [ ]:
## Здесь мы подсчитываем частоту использования трендов в улучшенных моделях. 

ss = s.sort_values(by='Разница R^2', ascending=False).reset_index(drop = True)
if var_name == 'nonfood_goods':
    columns = neprod.columns ## Выбираем тренды к целевой переменной
else: 
    columns = prod.columns 

for i in columns:
    s1 = pd.DataFrame()
    for j in range(len(ss['Регрессоры (тренды)'])):
        if i in ' '.join(ss['Регрессоры (тренды)'][j]): 
            s1 = s1.append(ss[ss.index == j])
    print(len(s1))

## Тесты Диболда-Мариано

Diebold-Mariano test  
Н0: Двусторонняя гипотеза о равенстве прогнозов  
Если p-value > 0.05, то гипотеза о равенстве прогнозов не отвергается  

Используем рекурсивное прогнозирование, заменяем лаги целевой переменной на оценки. Растущий трейн на каждом шаге помогает предсказать следующее наблюдение, т.е. здесь мы переоцениваем модель на каждом шаге.

In [ ]:
def match_func(index, lag, start, results, h):
    pattern = re.compile(var_name + index)
    match  = pattern.search(str(first_model1))
    if match:
        predictions_count = []
        start = results.fittedvalues.index[-1]
        for i in range(start, start+h+1):
            ### Назначаем текущее значение прогноза у его лагу.  
            X[var_name+index][i-lag+1] = results.predict(X[i:i+1])
            predictions_count.append(X[var_name+index][i-lag+1])
    return predictions_count

def get_predictions(h):
    try:
        predictions = match_func('-1', 1, results = results, h=h)
    except:
        try:
            predictions = match_func('-2', 2, results = results, h=h)
        except:
            try:
                predictions = match_func('-3', 3, results = results, h=h)
            except:
                predictions = []
                for i in range(start, start+h+1):
                     ## добавляем прогнозное наблюдение
                    predictions.append(results.predict(X[i:i+h+1]))
    return predictions

In [ ]:
DM = pd.DataFrame()
s = s.reset_index(drop = True)
for h in range(10):
    dm = []
    for j in range(len(s)):
        first_model1 = s.iloc[j,:]['Регрессоры'].split(', ')
        X = tab_2014.filter(first_model1) 
        y = tab_2014.filter([var_name])
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')

        start = results.fittedvalues.index[-1]
        predictions1 = []
        for i in range(start, y.index[-1]-h):
            predictions = get_predictions(h)
            predictions1.append(predictions[0][start+h])
            start +=1 
            X = tab_2014.filter(first_model1)  
            y = tab_2014.filter([var_name])
            X, y, n, t = result(X, y)
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')
    
        first_model1 = s.iloc[j,:]['Регрессоры'].split(', ')
        X_star = tab_2014.filter(first_model1)
        X = X_star.join(goods[s.iloc[j,4][len(s.iloc[j,0].split()):]])
        y = tab_2014.filter([var_name])
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')

        start = results.fittedvalues.index[-1]
        predictions2 = []
        for i in range(start, y.index[-1]-h):
            predictions = get_predictions(h)
            predictions2.append(predictions[0][start+h])
            start +=1 
            X = X_star.join(goods[s.iloc[j,4][len(s.iloc[j,0].split()):]])
            y = tab_2014.filter([var_name])
            X, y, n, t = result(X, y)
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')

        dm.append(dm_test(y[n:][var_name][h:], predictions2, predictions1, h = h+1, crit = 'MSE')[1])


    df = pd.DataFrame()
    for i in range(len(dm)):
        if dm[i]<0.2:
            df = df.append(s.iloc[i,:])
    df['Шаг прогноза'] = h+1
    DM = DM.append(df)

In [ ]:
df1 = DM[['Регрессоры (тренды)', 'Шаг прогноза']]
df2 = df1.groupby(level=0)['Шаг прогноза'].apply(list).to_dict()
keys = list(df2.keys())
names_ = []
for i in s.index:
    if i in keys:
        names_.append(str(s['Регрессоры (тренды)'][i]))

print('Количество уникальных регрессий ', len(pd.DataFrame([names_, df2.values()]).T))

## Подсчет количества моделей с шагом прогноза от 1 до 10.
#########################################################

for i in range(1,11):
    print(len(DM[DM['Шаг прогноза'] == i]))

In [ ]:
## Список моделей для конкретного шага прогноза 
###############################################

temp = DM[DM['Шаг прогноза'] == 10].sort_values(['Разница R^2'], ascending = False)
temp = temp.reset_index(drop = True)
temp